# Numerical differentiation of unequally sampled data using Machine Learning
- _Pascal Tribel - Machine Learning Group_
- _December 2024_
## Abstract
Derivative estimation is a fundamental problem in numerical analysis, scientific computing, and machine learning, with applications ranging from solving differential equations to data analysis in physics and engineering. Classical approaches to derivative estimation include finite difference methods, spline-based approximations, and spectral methods. These techniques, while effective in many settings, often face limitations when dealing with irregularly spaced (non-evenly discretized) input data, noise, or highly nonlinear functions.

Neural networks and neural operators have recently emerged as powerful tools for learning complex functional mappings. Neural operators, in particular, extend traditional neural network frameworks by operating in infinite-dimensional function spaces, making them highly suitable for learning mappings between functions. This framework has been successfully applied to problems such as solving partial differential equations (PDEs) (e.g., Fourier neural operators) and emulating physical systems. However, the application of neural operators for derivative estimation has not been thoroughly explored, especially for irregularly spaced input intervals.

Since the differentiation is commonly formulated using the operator paradigm, but the task can also be defined in the functional paradigm when facing sampled data, we define a framework to compare the performances of different Machine Learning architecture, while comparing them to classical numerical solvers.
## Problem statement
### Introduction and analytical method
In the analytical formulation, the definition of the differential operator of an univariate function is trivial:
$$\frac{df(x)}{dx} = \lim_{\delta x\rightarrow 0}\frac{f(x+\delta x) - f(x)}{\delta x} $$
This can in many cases easily be computed if the analytical form of $f(x)$ is known. Once the derivative $f'(x) = \frac{df(x)}{dx}$ is defined, it can be queried at any point. The value obtained denotes the instant slope of the initial function. Therefore, two formalisms can be used: the _operator_ $\frac{d}{dx}$ defines a mapping between two functions spaces, but there also may exists a function $g(x, \hat{f}) = f'(x)$ which computes the mapping between $x$ and the value of the derivative $f'(x)$ given numerical additional information on $f$ denoted by $\hat{f}$. This second formalism defines a mapping between two vector spaces.
### Numerical approaches
However, when dealing with sampled data (e.g. from real-world measurements), this analytical method does not work. Multiple approaches have been proposed, including the (2) forward, (3) central and (4) backward difference methods, such as, given three sampled points $x_0, x_1, x_2$ such that $x_0<x_1<x_2$:
$$ f'(x_1) = \frac{f(x_2) - f(x_1)}{x_2 - x_1}$$
$$ f'(x_1) = \frac{f(x_2) - f(x_0)}{x_2 - x_0}$$
$$ f'(x_1) = \frac{f(x_1) - f(x_0)}{x_1 - x_0}$$
Only the central difference method can be used in order to compute the derivative of a point $\hat{x}$ for which the function has not been sampled, by querying the closest left and right point to $\hat{x}$.
Such an approach often suffers when the data sampling is unequally spaced. To solve this problem, it is possible to proceed in three steps:
- Interpolate a spline between each k-uplet of neighboring points
- Compute the derivative of this spline
- Query the value of this derivative for any point in the interval bounded by the $k$ points
This approach may suffer by two means: either if the sampling resolution is not sufficient, or if the spline order does not correctly fits (overfits or underfits) the non-linearity of the data.
### Experimental approach

In [1]:
import numpy as np
import torch
import torch.nn as nn
import neuralop
from neuralop.models import FNO1d
import matplotlib.pyplot as plt
import seaborn as sns
import symengine as se
import pandas as pd
from tqdm.auto import tqdm
from copy import deepcopy
from joblib import Parallel, delayed
import scipy

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
def error(x, y):
    return np.mean(np.abs(x-y))

def diff_fwd(y, x): 
    return np.diff(y)/np.diff(x)

def diff_central(y, x):
    return np.array(np.gradient(y))[:-1]/np.array(np.gradient(x))[:-1]

def diff_interpolate(y, x, q):
    if len(q.shape)==1:
        return scipy.interpolate.UnivariateSpline(np.array(x), np.array(y), k=5).derivative()(np.array(q))[:-1]
    else:
        return np.array([scipy.interpolate.UnivariateSpline(np.array(x[i]), np.array(y[i]), k=5).derivative()(np.array(q[i]))[:-1] for i in range(q.shape[0])])
        
def random_transform(f, x):
    return f((10*float(np.random.random() - 0.5)) * x + (float(np.random.random() - 0.5))) + (float(np.random.random() - 0.5))

def random_polynom(x):
    degree = np.random.randint(2, 3)
    expression = (float(np.random.random() - 0.5)) * x**degree
    degree -= 1
    while degree >= 0:
        expression += 0.01*(float(np.random.random() - 0.5)) * x**degree
        degree -= 1
    return expression

def random_sum(l):
    res = []
    for f1 in l:
        res.append((float(np.random.random() - 0.5)) * f1 + (float(np.random.random() - 0.5)) * np.random.choice(l))
    return res

def random_product(l):
    res = []
    for f1 in l:
        res.append((float(np.random.random() - 0.5)) * f1 * (float(np.random.random() - 0.5)) * np.random.choice(l))
    return res

def random_function(x, input_num_points, output_num_points, input_length, output_length, noise=1e-2):
    maximum = 10
    minimum = -10
    while maximum >= 1 or minimum <= -1 or np.abs(np.max(f_eval)-np.min(f_eval)) <= 10e-2:
        bases = np.random.choice([random_transform(se.sin, x),
                                  random_transform(se.cos, x),
                                  random_transform(se.tanh, x),
                                  random_polynom(x)
                                 ], size=np.random.randint(1, 5))
        f = bases[0]
        for i in range(1, len(bases)):
            p = np.random.random()
            if p < 0.5:
                f = 0.1*random_sum([f, bases[i]])[0]
            else:
                f = 0.1*random_product([f, bases[i]])[0]
        if np.random.random() < 0.5:
            f = f * se.sin(25*float(np.random.random() - 0.5) * x)
        if np.random.random() < 0.5:
            f = f * se.cos(25*float(np.random.random() - 0.5) * x)
        if np.random.random() < 0.5:
            f = f * (se.tanh(x + float(np.random.random() - 0.5)) - 0.5) * float(np.random.random() - 0.5)
        df = se.diff(f, x)
        
        start = np.random.random()-1
        end = start + length
        input_points = np.sort(np.random.uniform(start, end, size=input_num_points))
        output_points = np.sort(np.random.uniform(start, end, size=output_num_points))
        f_eval = np.array([float(f.subs({x: p})) for p in input_points])
        
        df_eval = np.array([float(df.subs({x: p})) for p in output_points])
        maximum = np.max([np.max(f_eval), np.max(df_eval)])
        minimum = np.min([np.min(f_eval), np.min(df_eval)])
    return f, df, input_points, output_points, f_eval + np.random.normal(0, noise, size=input_num_points), df_eval

def generate_sample(x, input_num_points, output_num_points, length, noise):
    f, df, i_p, o_p, f_e, df_e = random_function(x, input_num_points, output_num_points, length, noise)
    return f, df, i_p, o_p, f_e, df_e

In [3]:
total_size = int(2**15)
length = 2
input_num_points = 64
output_num_points = 64
noise = 25e-2

In [14]:
GENERATING = False

if GENERATING:
    print("Generating", total_size, "samples")
    x = se.Symbol('x')

    results = Parallel(n_jobs=-1)(delayed(generate_sample)(x, input_num_points, output_num_points, length, noise) for _ in tqdm(range(total_size)))

    functions, derivatives, input_points, output_points, functions_eval, derivatives_eval = zip(*results)

    X = np.array(list(zip(functions_eval, input_points)))
    Y = np.array(list(zip(derivatives_eval, output_points)))
    np.save("X.npy", X)
    np.save("Y.npy", Y)
else:
    X, Y = np.load("X.npy"), np.load("Y.npy")
    total_size, _, num_points = X.shape

In [15]:
train_size = int(X.shape[0]*0.5)
val_size = int(X.shape[0]*0.25)
test_size = X.shape[0] - train_size - val_size

x_train, y_train = torch.tensor(X[:train_size], dtype=torch.float32).to('mps'), torch.tensor(Y[:train_size], dtype=torch.float32).to('mps')
x_val, y_val = torch.tensor(X[train_size:train_size+val_size], dtype=torch.float32).to('mps'), torch.tensor(Y[train_size:train_size+val_size], dtype=torch.float32).to('mps')
x_test, y_test = torch.tensor(X[train_size+val_size:], dtype=torch.float32).to('mps'), torch.tensor(Y[train_size+val_size:], dtype=torch.float32).to('mps')

In [16]:
maxi_0 = x_train[:, 0].max()
mini_0 = x_train[:, 0].min()
maxi_1 = x_train[:, 1].max()
mini_1 = x_train[:, 1].min()

x_train[:, 0] = (x_train[:, 0]-mini_0)/(maxi_0-mini_0)
x_train[:, 1] = (x_train[:, 1]-mini_1)/(maxi_1-mini_1)

x_val[:, 0] = (x_val[:, 0]-mini_0)/(maxi_0-mini_0)
x_val[:, 1] = (x_val[:, 1]-mini_1)/(maxi_1-mini_1)

x_test[:, 0] = (x_test[:, 0]-mini_0)/(maxi_0-mini_0)
x_test[:, 1] = (x_test[:, 1]-mini_1)/(maxi_1-mini_1)

maxi_0_y = y_train[:, 0].max()
mini_0_y = y_train[:, 0].min()
maxi_1_y = y_train[:, 1].max()
mini_1_y = y_train[:, 1].min()

y_train[:, 0] = (y_train[:, 0]-mini_0_y)/(maxi_0_y-mini_0_y)
y_train[:, 1] = (y_train[:, 1]-mini_1_y)/(maxi_1_y-mini_1_y)

y_val[:, 0] = (y_val[:, 0]-mini_0_y)/(maxi_0_y-mini_0_y)
y_val[:, 1] = (y_val[:, 1]-mini_1_y)/(maxi_1_y-mini_1_y)

y_test[:, 0] = (y_test[:, 0]-mini_0_y)/(maxi_0_y-mini_0_y)
y_test[:, 1] = (y_test[:, 1]-mini_1_y)/(maxi_1_y-mini_1_y)

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(NeuralNetwork, self).__init__()
        self.l1 = nn.Linear(input_shape, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 128)
        self.l4 = nn.Linear(128, output_shape)
        
        self.relu = nn.ReLU()

    def forward(self, inputs, query):
        x = self.relu(self.l1(torch.concat([inputs.reshape((inputs.shape[0], -1)), query.reshape((query.shape[0], -1))], axis=1)))
        x = self.relu(self.l2(x))
        x = self.relu(self.l3(x))
        x = self.l4(x)
        return x

class CNN(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * input_shape//4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_shape)
        
        self.relu = nn.ReLU()

    def forward(self, inputs, query):
        x = self.relu(self.conv1(torch.concat([inputs.reshape((inputs.shape[0], 1, -1)), query.reshape((query.shape[0], 1, -1))], axis=2)))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class NeuralOperator(nn.Module):
    def __init__(self, n_modes_height=64, n_layers=1, hidden_channels=16):
        super(NeuralOperator, self).__init__()
        self.fc = neuralop.models.FNO1d(n_modes_height=n_modes_height, n_layers=n_layers, in_channels=3, out_channels=1, hidden_channels=hidden_channels)

    def forward(self, inputs, query):
        inputs = inputs.view(inputs.size(0), -1, query.size(-1))
        return self.fc(torch.concat([inputs, query.unsqueeze(1)], 1)).squeeze(1)

In [20]:
torch.manual_seed(42)
neural_network = NeuralNetwork(input_num_points*2+output_num_points, output_num_points).to('mps')
print("Neural Network:", np.sum([len(i) for i in neural_network.parameters()]), "parameters")

Neural Network: 1920 parameters


In [21]:
torch.manual_seed(42)
cnn = CNN(input_num_points*2+output_num_points, output_num_points).to('mps')
print("Convolutional Neural Network:", np.sum([len(i) for i in cnn.parameters()]), "parameters")

Convolutional Neural Network: 1888 parameters


In [22]:
torch.manual_seed(42)
neural_operator = NeuralOperator(128, 4, 64).to('mps')
print("Neural Operator:", np.sum([len(i) for i in neural_operator.parameters()]), "parameters")

Neural Operator: 1670 parameters


In [23]:
def train(model, n_epochs, batch_size):
    best_model = deepcopy(model)
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    val_losses = []
    best_loss = np.inf
    for epoch in (pbar := tqdm(range(n_epochs))):
        model.train()
        for b in range(0, x_train.shape[0], batch_size):
            optimizer.zero_grad()
            predictions = model(x_train[b:b+batch_size], y_train[b:b+batch_size, 1])
            loss = criterion(predictions, y_train[b:b+batch_size, 0])
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(x_val, y_val[:, 1]), y_val[:, 0]).item()
        val_losses.append(val_loss)
        scheduler.step(val_loss)
        pbar.set_description(f"Loss: {val_loss:.9f}")
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = deepcopy(model)
    return best_model, val_losses

In [24]:
epochs = 128
batch_size = 128

In [25]:
best_neural_operator, losses_neural_operator = train(neural_operator, epochs, batch_size)

  0%|          | 0/128 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
best_neural_network, losses_neural_network = train(neural_network, epochs, batch_size)

In [ ]:
best_cnn, losses_cnn = train(cnn, epochs, batch_size)

In [ ]:
with torch.no_grad():
    y_hat_nn = best_neural_network(x_test, y_test[:, 1]).to('cpu').squeeze().detach().numpy()
    y_hat_cnn = best_cnn(x_test, y_test[:, 1]).to('cpu').squeeze().detach().numpy()
    y_hat_no = best_neural_operator(x_test, y_test[:, 1]).to('cpu').squeeze().detach().numpy()
    
y_hat_scipy = diff_interpolate(((x_test[:, 0]+mini_0)*(maxi_0-mini_0)).to('cpu'), ((x_test[:, 1]+mini_1)*(maxi_1-mini_1)).to('cpu'), y_test[:, 1].to('cpu'))
y_hat_scipy = (y_hat_scipy-mini_0_y.item())/(maxi_0_y.item()-mini_0_y.item())

In [ ]:
r = 0
errors = {"NN": [], "CNN": [], "NO": [], "SP": []}
n_samples = 10
for j in range(n_samples):
    i = r + j
    fig, ax = plt.subplots(1, 5, figsize=(25, 4))
    for a in range(4):
        ax[a+1].plot(y_test[i, 1].to("cpu"), y_test[i, 0].to("cpu"), linestyle="--", label="f'(x)")
    ax[0].plot(x_test[i, 1].to("cpu"), x_test[i, 0].to("cpu"), label='f(x)')
    ax[1].plot(y_test[i, 1].to("cpu"), y_hat_nn[i], label='nn')
    ax[2].plot(y_test[i, 1].to("cpu"), y_hat_cnn[i], label='cnn')
    ax[3].plot(y_test[i, 1].to("cpu"), y_hat_no[i], label='no')
    ax[4].plot(y_test[i, 1, :-1].to("cpu"), y_hat_scipy[i], label='sp')
    ax[0].set_title('Input function')
    errors["NN"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_nn[i]))
    errors["CNN"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_cnn[i]))
    errors["NO"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_no[i]))
    errors["SP"].append(error(y_test[i, 0, :-1].to("cpu").numpy(), y_hat_scipy[i]))
    ax[1].set_title(f'Neural Network\nMSE: {errors["NN"][-1]:.7f}')
    ax[2].set_title(f'Convolutional Neural Network\nMSE: {errors["CNN"][-1]:.7f}')
    ax[3].set_title(f'Neural Operator\nMSE: {errors["NO"][-1]:.7f}')
    ax[4].set_title(f'Numerical differentiation by interpolation\nMSE: {errors["SP"][-1]:.7f}')
    for a in range(5):
        ax[a].set_ylim(-0.5, 1.5)
        ax[a].legend()
        ax[a].grid('on')
    plt.show()

In [ ]:
r = 0
errors = {"NN": [], "CNN": [], "NO": [], "SP": []}
n_samples = x_test.shape[0]
for j in range(n_samples):
    i = r + j
    errors["NN"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_nn[i]))
    errors["CNN"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_cnn[i]))
    errors["NO"].append(error(y_test[i, 0].to("cpu").numpy(), y_hat_no[i]))
    errors["SP"].append(error(y_test[i, 0, :-1].to("cpu").numpy(), y_hat_scipy[i]))
averages = {i: [np.mean(errors[i])] for i in errors}

In [ ]:
for i in sorted(averages, key=lambda x: averages[x]):
    print(f"{i}:\t{averages[i][0]:.7f}")

In [ ]:
errors = pd.DataFrame(errors)
averages = pd.DataFrame(averages)

In [ ]:
fig, ax = plt.subplots(figsize=(17, 10))
sns.stripplot(data=errors, ax=ax, marker='.')
sns.stripplot(data=averages, ax=ax, color='black', marker='o')
for i in averages:
    ax.plot([0, 3], [averages[i], averages[i]])
plt.legend(["Multi-layer Perceptron", "Convolutional Neural Network", "Neural Operator", "Interpolation differentiation"])
plt.title("Errors")
plt.grid('on')
plt.yscale('log')
plt.show()